In [1]:
import os
from tqdm import tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
pd.set_option('display.float_format', '{:.2f}'.format)
print(os.listdir("../input/eyn-original"))
x_min, x_max = -1., 1.
y_min, y_max = -.3, .3

['train_data.npy', 'trn_index_list.npy', 'custom.css', 'val_index_list.npy', '__results__.html', 'train_targets_inside.npy', 'submission.csv', 'train_targets.npy', 'test_ids.npy', '__output__.json', 'test_data.npy', '__notebook__.ipynb']


In [2]:
df_train = pd.read_pickle("../input/eyn-original-df/df_train.pickle")
df_test = pd.read_pickle("../input/eyn-original-df/df_test.pickle")
print(df_train.shape, df_test.shape)
df_train.head()

(2815323, 11) (703815, 11)


t_entry  t_exit  x_entry  y_entry  ...    vmin  vmean  tid_0  tid_1
seq traj                                     ...                             
0   0        0.00    0.03    -1.60    -0.43  ...     nan    nan   5.00   1.00
    1       -0.04   -0.04    -1.60    -0.47  ...     nan    nan   4.00   1.00
    2       -0.67   -0.64    -1.60    -0.53  ...     nan    nan   3.00   1.00
    3       -0.71   -0.69    -1.60    -0.55  ...     nan    nan   2.00   1.00
    4       -0.77   -0.76    -1.70    -0.84  ...     nan    nan   1.00   1.00

[5 rows x 11 columns]

In [3]:
df_train_values = df_train.values
df_train_values.shape

(2815323, 11)

In [4]:
train_targets = np.load("../input/eyn-original/train_targets.npy")
print(train_targets.shape)
train_targets_col_loc = [list(df_train.columns).index("x_exit"),
                         list(df_train.columns).index("y_exit")]
df_train_values[::21,train_targets_col_loc] = train_targets
df_train[df_train.columns] = df_train_values
df_train.head()

(134063, 2)


t_entry  t_exit  x_entry  y_entry  ...    vmin  vmean  tid_0  tid_1
seq traj                                     ...                             
0   0        0.00    0.03    -1.60    -0.43  ...     nan    nan   5.00   1.00
    1       -0.04   -0.04    -1.60    -0.47  ...     nan    nan   4.00   1.00
    2       -0.67   -0.64    -1.60    -0.53  ...     nan    nan   3.00   1.00
    3       -0.71   -0.69    -1.60    -0.55  ...     nan    nan   2.00   1.00
    4       -0.77   -0.76    -1.70    -0.84  ...     nan    nan   1.00   1.00

[5 rows x 11 columns]

In [5]:
def is_inside(arr_x, arr_y):
    inside = ((arr_x > x_min) & 
             (arr_x < x_max) & 
             (arr_y > y_min) & 
             (arr_y < y_max)).astype(float)
    inside[np.isnan(arr_x)] = np.nan  # replace nan position with nan judgement
    return inside

df_train['entry_in'] = is_inside(df_train['x_entry'], df_train['y_entry'])
df_train['exit_in'] = is_inside(df_train['x_exit'], df_train['y_exit'])
df_test['entry_in'] = is_inside(df_test['x_entry'], df_test['y_entry'])
df_test['exit_in'] = is_inside(df_test['x_exit'], df_test['y_exit'])
df_train.head()

t_entry  t_exit  x_entry   ...     tid_1  entry_in  exit_in
seq traj                             ...                             
0   0        0.00    0.03    -1.60   ...      1.00      0.00     0.00
    1       -0.04   -0.04    -1.60   ...      1.00      0.00     0.00
    2       -0.67   -0.64    -1.60   ...      1.00      0.00     0.00
    3       -0.71   -0.69    -1.60   ...      1.00      0.00     0.00
    4       -0.77   -0.76    -1.70   ...      1.00      0.00     0.00

[5 rows x 13 columns]

In [6]:
def calc_diff_alt(arr_entry, arr_exit):
    return np.append(np.array(arr_entry)[:-1] - np.array(arr_exit)[1:], np.nan)

df_train['dur'] = df_train['t_exit'] - df_train['t_entry']
df_test['dur'] = df_test['t_exit'] - df_test['t_entry']
df_train['dur_a'] = calc_diff_alt(df_train['t_entry'], df_train['t_exit'])
df_test['dur_a'] = calc_diff_alt(df_test['t_entry'], df_test['t_exit'])
df_test.head()

t_entry  t_exit  x_entry  ...    exit_in  dur  dur_a
seq traj                            ...                       
0   0        0.01    0.02     1.22  ...        nan 0.01   0.13
    1       -0.16   -0.13     1.25  ...       0.00 0.03   0.02
    2       -0.24   -0.18     0.29  ...       0.00 0.07   0.02
    3       -0.26   -0.26     1.23  ...       0.00 0.00   0.05
    4       -0.33   -0.32     1.25  ...       0.00 0.01    nan

[5 rows x 15 columns]

In [7]:
def calc_dist(x1, x2, y1, y2):
    x1, x2, y1, y2 = [np.array(arr) for arr in [x1, x2, y1, y2]]
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    
df_train['dist'] = calc_dist(df_train['x_entry'], df_train['x_exit'], df_train['y_entry'], df_train['y_exit'])
df_test['dist'] = calc_dist(df_test['x_entry'], df_test['x_exit'], df_test['y_entry'], df_test['y_exit'])
df_train['dist_a'] = np.append(calc_dist(df_train['x_entry'][:-1], df_train['x_exit'][1:], 
                                         df_train['y_entry'][:-1], df_train['y_exit'][1:]),np.nan)
df_test['dist_a'] = np.append(calc_dist(df_test['x_entry'][:-1], df_test['x_exit'][1:], 
                                        df_test['y_entry'][:-1], df_test['y_exit'][1:]),np.nan)
df_train.head()

t_entry  t_exit  x_entry  y_entry   ...    dur  dur_a  dist  dist_a
seq traj                                      ...                            
0   0        0.00    0.03    -1.60    -0.43   ...   0.03   0.04  0.02    0.04
    1       -0.04   -0.04    -1.60    -0.47   ...   0.00   0.60  0.00    0.05
    2       -0.67   -0.64    -1.60    -0.53   ...   0.03   0.02  0.02    0.01
    3       -0.71   -0.69    -1.60    -0.55   ...   0.02   0.05  0.01    0.26
    4       -0.77   -0.76    -1.70    -0.84   ...   0.01   0.02  0.11    1.97

[5 rows x 17 columns]

In [8]:
def nan_divide(numerator, denominator):
    result = np.divide(numerator, denominator, 
                       out=np.zeros_like(denominator), where=denominator!=0)
    result[np.isnan(numerator)] = np.nan
    return result

df_train['speed'] = nan_divide(df_train['dist'], df_train['dur'])
df_test['speed']= nan_divide(df_test['dist'], df_test['dur'])
df_train['speed_a'] = nan_divide(df_train['dist_a'], df_train['dur_a'])
df_test['speed_a'] = nan_divide(df_test['dist_a'], df_test['dur_a'])
df_train.head()

t_entry  t_exit  x_entry   ...     dist_a  speed  speed_a
seq traj                             ...                           
0   0        0.00    0.03    -1.60   ...       0.04   0.61     0.93
    1       -0.04   -0.04    -1.60   ...       0.05   0.00     0.08
    2       -0.67   -0.64    -1.60   ...       0.01   0.60     0.35
    3       -0.71   -0.69    -1.60   ...       0.26   0.54     5.63
    4       -0.77   -0.76    -1.70   ...       1.97  12.56    98.12

[5 rows x 19 columns]

In [9]:
df_train['dir_x'] = nan_divide(df_train['x_exit'] - df_train['x_entry'], df_train['dist'])
df_train['dir_y'] = nan_divide(df_train['y_exit'] - df_train['y_entry'], df_train['dist'])
df_test['dir_x'] = nan_divide(df_test['x_exit'] - df_test['x_entry'], df_test['dist'])
df_test['dir_y'] = nan_divide(df_test['y_exit'] - df_test['y_entry'], df_test['dist'])
df_train['dir_x_a'] = nan_divide(calc_diff_alt(df_train['x_entry'], df_train['x_exit']), df_train['dist_a'])
df_train['dir_y_a'] = nan_divide(calc_diff_alt(df_train['y_entry'], df_train['y_exit']), df_train['dist_a'])
df_test['dir_x_a'] = nan_divide(calc_diff_alt(df_test['x_entry'], df_test['x_exit']), df_test['dist_a'])
df_test['dir_y_a'] = nan_divide(calc_diff_alt(df_test['y_entry'], df_test['y_exit']), df_test['dist_a'])
df_train.head()

t_entry  t_exit  x_entry   ...     dir_y  dir_x_a  dir_y_a
seq traj                             ...                            
0   0        0.00    0.03    -1.60   ...      0.22     0.09     1.00
    1       -0.04   -0.04    -1.60   ...      0.00     0.20     0.98
    2       -0.67   -0.64    -1.60   ...      0.93     0.76     0.65
    3       -0.71   -0.69    -1.60   ...      0.81    -0.04     1.00
    4       -0.77   -0.76    -1.70   ...      0.26    -0.32    -0.95

[5 rows x 23 columns]

In [10]:
df_train.to_pickle("df_train_full_targets.pickle")
df_test.to_pickle("df_test.pickle")

In [11]:
# to load
df_train = pd.read_pickle("df_train_full_targets.pickle")
df_test = pd.read_pickle("df_test.pickle")
print(df_train.shape, df_test.shape)

(2815323, 23) (703815, 23)


In [12]:
df_train_values = df_train.values
train_targets_full_col = [list(df_train.columns).index("x_exit"),
                          list(df_train.columns).index("y_exit"),
                          list(df_train.columns).index("dist"),
                          list(df_train.columns).index("speed"),
                          list(df_train.columns).index("dir_x"),
                          list(df_train.columns).index("dir_y")]
train_targets_full = df_train_values[::21,train_targets_full_col]
print(train_targets_full.shape)
np.save("train_targets_full", train_targets_full)

(134063, 6)
